In [1]:
!jupyter nbextension enable codefolding/main


Enabling notebook extension codefolding/main...
      - Validating: problems found:
        - require?  X codefolding/main


In [2]:
import pandas as pd
import numpy as np
from ast import literal_eval

In [3]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 199)
pd.options.display.float_format = '{:.2f}'.format

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# Overview
1. Identify US users 
2. Identify US Tweets 
3. Delete all obs. without US User or Tweet 
4. Match to list of users we collected

## Import GeoCov19 Data

In [4]:
df = pd.read_json(r"C:/Users/crackcocaine69xxx/Python Stuff/594/GeoCoV19 Data/geo_feb_01_10/geo_2020-02-01/geo_2020-02-01.json", lines=True)

# DELETE THIS LATER

In [5]:
df = df.iloc[:5000]

### Make Uppercase

In [106]:
def make_dict_uppercase(d):
    '''
    input = dictionary
    output = uppercase dictionary
    '''
    
    return {key.upper(): value.upper() for key,value in d.items()}

df['user_location'] = df['user_location'].apply(make_dict_uppercase)


##################################################################################


def make_list_of_dicts_uppercase(list_of_dicts):
    '''
    input = list of dictionaries
    output = uppercase list of dictionaries
    '''
    
    return [make_dict_uppercase(d) for d in list_of_dicts]


df['tweet_locations'] = df['tweet_locations'].apply(make_list_of_dicts_uppercase)

## Get (State & County) --> FIPS dictionary

In [107]:
# import data from wikipedia
fips = pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_FIPS_codes_by_county")[1]

# remove all hyperlinks (these look like "... County [h]", etc.)
fips['County or equivalent'] = fips['County or equivalent'].str.replace(r"\[.*\]","")

# convert to uppercase
fips['County or equivalent'] = fips['County or equivalent'].apply(lambda x: x.upper())
fips['State or equivalent'] = fips['State or equivalent'].apply(lambda x: x.upper())

# replace "St." with "Saint"
fips['County or equivalent'] = [x.replace('ST.','SAINT') for x in fips['County or equivalent']]

# remove everything after a comma in a county name (e.g. "ANCHORAGE, MUNICIPALITY OF")
fips['County or equivalent'] = [x.split(',')[0] for x in fips['County or equivalent']]

# replace DC info to correspond to GeoCov19 format
dc_loc = fips[fips['County or equivalent']=='DISTRICT OF COLUMBIA'].index.tolist()[0]
fips['State or equivalent'].loc[dc_loc] = 'WASHINGTON, D.C.'
fips['County or equivalent'].loc[dc_loc] = 'WASHINGTON'

########################################################################

# create dictionary
state_fips_dict = {k: f.groupby('County or equivalent')['FIPS'].apply(list).to_dict()
     for k, f in fips.groupby('State or equivalent')}

# clean dictionary
for state in state_fips_dict:
    for county in state_fips_dict[state]:
        state_fips_dict[state][county] = state_fips_dict[state][county][0]

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


# Map county names to FIPS (using FIPS dict)

In [127]:
def get_fips_from_loc(loc):
    '''
    input = tweet_location from GeoCov19 data (single dictionary)
    output = FIPS code corresponding to counties mentioned
    '''

    if loc['COUNTRY_CODE']=='US':

        try:

            if loc['COUNTY'].split(" ")[-1] in ('COUNTY', 'PARISH'):

                return state_fips_dict[ loc['STATE'] ] [loc['COUNTY'] ]

            elif 'COUNTY' in loc:

                try:

                    return state_fips_dict[ loc['STATE'] ] [loc['COUNTY'] + " " + "COUNTY"]

                except Exception as e:

                    pass
        
        except Exception as e:

            pass

In [122]:
def get_fips_from_list_of_locs(tweet_locs):
    '''
    input = tweet_locations from GeoCov19 data (list of dicts)
    output = list of FIPS codes corresponding to counties mentioned
    '''

    temp_list = [get_fips_from_loc(loc, tweet_locs) for loc in tweet_locs]
    
    return [x for x in temp_list if x is not None]

In [110]:
# def OG_get_fips_from_list_of_locs(tweet_locs):
#     '''
#     input = tweet_locations from GeoCov19 data (list of dicts)
#     output = list of FIPS codes corresponding to counties mentioned
#     '''
#     tweet_FIPS = []
    
#     for loc in tweet_locs: #there are no longer any empty locs so we don't have an if-else for them
        
#         if loc['COUNTRY_CODE']=='US':

#             try:
                
#                 if loc['COUNTY'].split(" ")[-1] in ('COUNTY', 'PARISH'):
                    
#                     tweet_FIPS.append(state_fips_dict[ loc['STATE'] ] [loc['COUNTY'] ])
                                
#                 elif 'COUNTY' in loc:
                    
#                     try:
                        
#                         tweet_FIPS.append(state_fips_dict[ loc['STATE'] ] [loc['COUNTY'] + " " + "COUNTY"])
                        
#                     except Exception as e:
                        
#                         print('0 ', e, '\n')
#                         print(loc, '\n')
#                         print(tweet_locs, '\n \n')
                    
#             except Exception as e:
                
#                 try:
                    
#                     print((loc['STATE'], loc['COUNTY']),',', '\n')
#                     print('1 ', e, '\n')
#                     print(loc, '\n')
#                     print(tweet_locs, '\n')
#                     print(state_fips_dict[loc['STATE']], '\n')
#                     print(state_fips_dict[loc['STATE']][loc['COUNTY']], '\n')
#                     print('\n \n')
                    
#                 except:
                    
#                     pass
#                     print('2 ', e, '\n')
#                     print(loc, '\n')
#                     print(tweet_locs, '\n \n')

#     return tweet_FIPS

## NEED TO INSPECT COUNTY MAPPINGS TO ENSURE THAT IT PICKS UP ON STRINGS WELL


# ALL REPLACEMENTS MUST BE DONE IN "fips"
* ~"Pointe Coupee Parish County" should be "Pointe Coupee Parish"~
* ~Need to replace "St." with "Saint" in "fips"~
* Fix counties that aren't cross-referenced correctly:
    * "D.C."
    * "SAN FRANCISCO"

In [123]:
df['Tweet FIPS'] = df['tweet_locations'].apply(get_fips_from_list_of_locs)

0  'WASHINGTON COUNTY' 

{'COUNTRY_CODE': 'US', 'STATE': 'WASHINGTON, D.C.', 'COUNTY': 'WASHINGTON', 'CITY': 'WASHINGTON D.C.'} 

[{'COUNTRY_CODE': 'US', 'STATE': 'MARYLAND', 'COUNTY': 'DORCHESTER COUNTY', 'CITY': 'SECRETARY'}, {'COUNTRY_CODE': 'FR', 'STATE': 'AUVERGNE-RHÔNE-ALPES', 'COUNTY': 'ANNECY', 'CITY': 'ALEX'}, {'COUNTRY_CODE': 'IL', 'STATE': 'HAIFA DISTRICT', 'CITY': 'HAIFA'}, {'COUNTRY_CODE': 'US', 'STATE': 'TEXAS', 'COUNTY': 'TRINITY COUNTY'}, {'COUNTRY_CODE': 'GB', 'STATE': 'ENGLAND', 'COUNTY': 'DERBYSHIRE'}, {'COUNTRY_CODE': 'LR', 'STATE': 'GRAND BASSA COUNTY', 'COUNTY': 'DISTRICT # 2'}, {'COUNTRY_CODE': 'NZ', 'STATE': 'MARLBOROUGH'}, {'COUNTRY_CODE': 'EE', 'COUNTY': 'MUHU VALD'}, {'COUNTRY_CODE': 'GB', 'STATE': 'ENGLAND', 'COUNTY': 'DERBYSHIRE'}, {'COUNTRY_CODE': 'US', 'STATE': 'WASHINGTON, D.C.', 'COUNTY': 'WASHINGTON', 'CITY': 'WASHINGTON D.C.'}] 
 

0  'CITY OF SAINT LOUIS COUNTY' 

{'COUNTRY_CODE': 'US', 'STATE': 'MISSOURI', 'COUNTY': 'CITY OF SAINT LOUIS', 'CITY': 'S

In [125]:
df['User FIPS'] = df['user_location'].apply(get_fips_from_loc)

TypeError: get_fips_from_loc() missing 1 required positional argument: 'tweet_locs'

In [126]:
df.iloc[0]

,tweet_id,created_at,user_id,geo_source,user_location,geo,place,tweet_locations,FIPS
0,1223489703456006144,2020-02-01 06:14:11+00:00,30746211,user_location,{'COUNTRY_CODE': 'CA'},{},{},"[{'COUNTRY_CODE': 'UA', 'STATE': 'ZHYTOMYR OBLAST', 'COUNTY': 'SLOVECHNE AMALGAMATED TERRITORIAL COMMUNITY'}, {'COUNTRY_CODE': 'ES', 'STATE': 'ANDALUSIA', 'COUNTY': 'JAÉN', 'CITY': 'JAÉN'}, {'COU...","[40139, 24035]"
1,1223489798142230528,2020-02-01 06:14:33+00:00,2411706499,tweet_text,{},{},{},"[{'COUNTRY_CODE': 'RU', 'STATE': 'TATARSTAN', 'COUNTY': 'VYSOKOGORSKY DISTRICT'}, {'COUNTRY_CODE': 'US', 'STATE': 'NEW YORK', 'COUNTY': 'NEW YORK COUNTY', 'CITY': 'NEW YORK'}]",[36061]
2,1223489799044001792,2020-02-01 06:14:34+00:00,884746422364418048,user_location,"{'COUNTRY_CODE': 'CN', 'STATE': 'HONG KONG'}",{},{},"[{'COUNTRY_CODE': 'SG'}, {'COUNTRY_CODE': 'GB', 'STATE': 'ENGLAND', 'COUNTY': 'EAST SUSSEX', 'CITY': 'ROTHER'}, {'COUNTRY_CODE': 'YE', 'STATE': 'DHAMAR GOVERNORATE', 'COUNTY': 'ANSS DISTRICT'}, {...","[51051, 27071]"
3,1223489811459108864,2020-02-01 06:14:37+00:00,29512878,user_location,"{'COUNTRY_CODE': 'PH', 'STATE': 'CAVITE', 'CITY': 'DASMARINAS'}",{},{},[],[]
4,1223489829817577472,2020-02-01 06:14:41+00:00,3244922072,user_location,"{'COUNTRY_CODE': 'TH', 'STATE': 'SARABURI PROVINCE'}",{},{},[],[]
...,...,...,...,...,...,...,...,...,...
4995,1223744586633240576,2020-02-01 23:07:00+00:00,20442930,user_location,"{'COUNTRY_CODE': 'GB', 'STATE': 'ENGLAND', 'CITY': 'LONDON'}",{},{},"[{'COUNTRY_CODE': 'GB', 'STATE': 'ENGLAND', 'COUNTY': 'CAMBRIDGESHIRE', 'CITY': 'CAMBRIDGE'}]",[]
4996,1223499618123763712,2020-02-01 06:53:35+00:00,2334165923,user_location,"{'COUNTRY_CODE': 'CA', 'STATE': 'ONTARIO'}",{},{},"[{'COUNTRY_CODE': 'FR', 'STATE': 'NEW AQUITAINE', 'COUNTY': 'SARLAT-LA-CANÉDA', 'CITY': 'BARS'}, {'COUNTRY_CODE': 'CN'}]",[]
4997,1223499625753206784,2020-02-01 06:53:36+00:00,423168084,user_location,"{'COUNTRY_CODE': 'US', 'STATE': 'NEW YORK'}",{},{},"[{'COUNTRY_CODE': 'RU', 'STATE': 'VORONEZH OBLAST', 'COUNTY': 'PETROPAVLOVSKY DISTRICT'}, {'COUNTRY_CODE': 'FR', 'STATE': 'NEW AQUITAINE', 'COUNTY': 'MONT-DE-MARSAN', 'CITY': 'BATS'}]",[]
4998,1223499637455314944,2020-02-01 06:53:39+00:00,2216315264,user_location,{'COUNTRY_CODE': 'LB'},{},{},"[{'COUNTRY_CODE': 'FR', 'STATE': 'AUVERGNE-RHÔNE-ALPES', 'COUNTY': 'THONON-LES-BAINS', 'CITY': 'NOVEL'}, {'COUNTRY_CODE': 'US', 'STATE': 'ARKANSAS', 'COUNTY': 'CROSS COUNTY'}, {'COUNTRY_CODE': 'C...",[5037]
